# Transform_decrees

A partir da função _transform_legislations o texto será retornado com os tokens de decretos criados.

Essa função padroniza os decretos encontrados no texto, de forma que:

- Vide decreto-Lei nº 6.231 seja substituida por Vide ST_DECRETO_6231
- Decretos-federais n12,3 seja substituida por ST_DECRETO_123


In [1]:
import re

ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


In [2]:
def transform_decrees(text, **kwargs):
    #Create custom tokens for decrees appearances in a text

    decree_types = (
        "leis?",
        "regulamentador(?:es)?",
        "presidencia(?:l|is)",
        "federa(?:l|is)",
        "estadua(?:l|is)",
        "executivos?",
        "leis? complementar(?:es)?",
        "municipa(?:l|is)",
        "legislativos?",
        "legislativos? estadua(?:l|is)",
        "legislativos? municipa(?:l|is)",
    )
    decree_types = "|".join(decree_types)

    word_number = r"(?:n[oº]?\.?|n\.?º|n[úu]mero)"
    number = r"(?:\d+(?:[\.,]\d+)*)"
    year = fr"(?:[/-]{number})?"
    prefix_result = "DECRETO_"
    previous_token = fr"{ST_PREFIX}{prefix_result}{number}" #ST_DECRETO_
    separator = r"\s*[,e]\s+"
    decree = fr"(?P<identification>{number}){year}"
    re_legislation = re.compile(
        fr"(?P<ignored_expression>)(decretos?([\s-]+({decree_types}))?)\s+(?:{word_number}\s*)?{decree}",
        flags=re.I,
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        decree,
        previous_token,
        separator,
    )


In [7]:
entradas = """
  Vide Decreto-Lei nº 5.452, de 02 de Outubro de 1955
  Vide decreto-Lei nº 6.231
  art. 15 do Decreto-Lei no 2.848
  Artigo 4 do Decreto Lei nº 2.848 de 07 de Dezembro de 1940

"""

print("Entrada: ", entradas)

Entrada:  
  Vide Decreto-Lei nº 5.452, de 02 de Outubro de 1955
  Vide decreto-Lei nº 6.231
  art. 15 do Decreto-Lei no 2.848
  Artigo 4 do Decreto Lei nº 2.848 de 07 de Dezembro de 1940




In [8]:
print("Saida: ",transform_decrees(entradas))

Saida:  
  Vide ST_DECRETO_5452, de 02 de Outubro de 1955
  Vide ST_DECRETO_6231
  art. 15 do ST_DECRETO_2848
  Artigo 4 do ST_DECRETO_2848 de 07 de Dezembro de 1940




# DATASET VICTOR

In [3]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [4]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [5]:
lista = []
for item in text:
    lista.append(transform_decrees(item))

In [6]:
print("Dados de saida: \n", lista[12])

Dados de saida: 
 {"favorecido nem seja insignificante pela circunstância experimentada grifos nossos assim o ressarcimento pelo dano moral uma forma compensar mal causado não pode ser fonte enriquecimento de abusos devendo sua importância ser moderada que não foi atentado pelo juízo quo convém citar advertência desembargador tribunal justiça rio grande sul décio antonio erpen dano moral a desagregação social boletim coad adv informativo semanal pág propósito excesso pleito pode gerar esta situação a conseqüente reação pelas dimensões que assumir censura judicial objetivo reparação pecuniária não punir ofensor mas deferir uma sensação agradável vítima nunca uma fonte riqueza nem ato vindita grifos nossos melhor doutrina unânime afirmar que indenização por danos morais deve revestir de caráter eminentemente compensatório que jamais poderá servir fonte enriquecimento para lesado conforme ensinamento eminente professor humberto theodoro júnior sua obra dano moral nisso razão ordem ética q